In [40]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [41]:
# Get initial list: https://www.wikiwand.com/en/List_of_current_United_States_Senators
# For ease of manipulation, plug into Google Sheets and then export to .csv
# Remove unnecessary columns
# Add in needed columns and clean data
# BA University, BA University Public/Private, BA University White %, BA University Lat/Lng, DeVos Vote

def clean_INSTNM(school):
    ba = [school for school in school.split("\n") if "(B" in school]
    if len(ba) == 0: # John Boozeman
        ba = [school for school in school.split("\n") if "(O" in school]
    if len(ba) == 0: # Rand Paul
        ba = [school for school in school.split("\n") if "(M" in school]
    ba = ba[0].split("(")[0]
    if ba[-1] == " ":
        ba = ba[:-1]
    return ba

def clean_party(party):
    if party == "Independent[1]": # Bernie Sanders
        return "Independent"
    else:
        return party

# https://www.nytimes.com/interactive/2017/02/07/us/politics/betsy-devos-confirmation-vote.html
def devos(row):
    senator = row["Name"]
    party = row["Party"]
    if senator == "Lisa Murkowski" or senator == "Susan Collins":
        return "No"
    if party == "Democratic" or party == "Independent":
        return "No"
    else:
        return "Yes"
    
senators = pd.read_csv("senators.csv")
senators = senators[["State", "Name", "Party", "Education"]]
# CHANGE TO FUNCTION THAT ACCOUNTS FOR 'ATTENDED'
senators["INSTNM"] = senators["Education"].apply(lambda x: clean_INSTNM(x))
senators["Party"] = senators["Party"].apply(lambda x: clean_party(x))
senators["DeVos"] = senators.apply(lambda x: devos(x), axis=1)

senators.head()

,State,Name,Party,Education,INSTNM,DeVos
0,Alabama,Richard Shelby,Republican,"University of Alabama, Tuscaloosa(BA; LLB)\nBi...","University of Alabama, Tuscaloosa",Yes
1,Alabama,Luther Strange,Republican,"Tulane University(BA, JD)",Tulane University,Yes
2,Alaska,Lisa Murkowski,Republican,Georgetown University(BA)\nWillamette Universi...,Georgetown University,No
3,Alaska,Dan Sullivan,Republican,Harvard University(BA)\nGeorgetown University(...,Harvard University,Yes
4,Arizona,John McCain,Republican,United States Naval Academy (BS),United States Naval Academy,Yes


In [42]:
# University Data
# https://catalog.data.gov/dataset/college-scorecard

# "CONTROL": 1 = Public, 2 = Private nonprofit, 3 = Private for-profit
def clean_CONTROL(number):
    if number == 1:
        return "Public"
    elif number == 2:
        return "Private nonprofit"
    elif number == 3:
        return "Private for-profit"

universities = pd.read_csv("CollegeScorecard_Raw_Data/MERGED2014_15_PP.csv")
universities = universities[["INSTNM", "CITY", "STABBR", "ZIP", "LATITUDE", "LONGITUDE", "CONTROL", 
                             "UGDS", "UGDS_WHITE", "COSTT4_A"]]
universities["CONTROL"] = universities["CONTROL"].apply(lambda x: clean_CONTROL(x))
# "UG": Number of undergrads

universities.head()

,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,CONTROL,UGDS,UGDS_WHITE,COSTT4_A
0,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502,Public,4206,0.0333,21475
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.502230,-86.809170,Public,11383,0.5922,20621
2,Amridge University,Montgomery,AL,36117-3553,32.362609,-86.174010,Private nonprofit,291,0.2990,16370
3,University of Alabama in Huntsville,Huntsville,AL,35899,34.722818,-86.638420,Public,5451,0.6988,21107
4,Alabama State University,Montgomery,AL,36104-0271,32.364317,-86.295677,Public,4811,0.0158,18184


In [43]:
# Merge 'senators' with 'universities' on college name
# Because of formatting issues, let's make sure the names line up

def correct_instnm(previous):
    if previous == "Tulane University":
        return "Tulane University of Louisiana"
    if previous == "University of Alabama, Tuscaloosa":
        return "The University of Alabama"
    if previous == "University of Arkansas, Fayetteville":
        return "University of Arkansas"
    if previous == "Colorado State University, Fort Collins":
        return "Colorado State University-Fort Collins"
    if previous == "University of Wisconsin, Milwaukee":
        return "University of Wisconsin-Milwaukee"
    if previous == "Colorado State University, Fort Collins":
        return "Colorado State University-Fort Collins"
    if previous == "University of Hawaii, Manoa":
        return "University of Hawaii at Manoa"
    if previous == "University of Mississippi, Oxford":
        return "University of Mississippi"
    if previous == "Louisiana State University, Baton Rogue":
        return "Louisiana State University and Agricultural & Mechanical College"
    if previous == "Kansas State University, Manhattan":
        return "Kansas State University"
    if previous == "Georgia Institute of Technology":
        return "Georgia Institute of Technology-Main Campus"
    if previous == "Washington State University, Pullman":
        return "Washington State University"
    if previous == "West Virginia University, Morgantown":
        return "West Virginia University"
    if previous == "University of Minnesota, Twin Cities":
        return "University of Minnesota-Twin Cities"
    if previous == "University of Missouri, Columbia":
        return "University of Missouri-Columbia"
    if previous == "University of Tennessee, Knoxville":
        return "The University of Tennessee-Knoxville"
    if previous == "University of South Carolina, Columbia":
        return "University of South Carolina-Columbia"
    if previous == "University of California, Santa Barbara":
        return "University of California-Santa Barbara"
    if previous == "University of Texas, Austin":
        return "The University of Texas at Austin"
    if previous == "University of Nevada, Reno":
        return "University of Nevada-Reno"
    if previous == "University of Nebraska, Lincoln":
        return "University of Nebraska-Lincoln"
    if previous == "Montana State University, Bozeman":
        return "Montana State University"
    if previous == "Ohio State University":
        return "Ohio State University-Main Campus"
    if previous == "Trinity University, Texas":
        return "Trinity University"
    if previous == "Shippensburg University":
        return "Shippensburg University of Pennsylvania"
    if previous == "University of Pittsburgh":
        return "University of Pittsburgh-Pittsburgh Campus"
    if previous == "St. Lawrence University":
        return "St Lawrence University"
    if previous == "Brigham Young University, Utah":
        return "Brigham Young University-Provo"
    if previous == "Southwest Baptist University, Bolivar":
        return "Southwest Baptist University"
    if previous == "St. Michael's College":
        return "Saint Michael's College"
    if previous == "University of Idaho, Moscow":
        return "University of Idaho"
    if previous == "St. Peter's University":
        return "Saint Peter's University"
    if previous == "University of Kansas, Lawrence":
        return "University of Kansas"
    if previous == "University of Maryland, University College":
        return "University of Maryland-University College"
    if previous == "Miami University":
        return "Miami University-Oxford"
    else:
        return previous

senators["INSTNM"] = senators["INSTNM"].apply(lambda x: correct_instnm(x))
senator_univs = pd.merge(senators, universities, on="INSTNM", how="left")
senator_univs.head()

,State,Name,Party,Education,INSTNM,DeVos,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,CONTROL,UGDS,UGDS_WHITE,COSTT4_A
0,Alabama,Richard Shelby,Republican,"University of Alabama, Tuscaloosa(BA; LLB)\nBi...",The University of Alabama,Yes,Tuscaloosa,AL,35487-0166,33.214400,-87.545766,Public,29851,0.7825,26400
1,Alabama,Luther Strange,Republican,"Tulane University(BA, JD)",Tulane University of Louisiana,Yes,New Orleans,LA,70118-5698,29.939696,-90.120409,Private nonprofit,7892,0.7138,61055
2,Alaska,Lisa Murkowski,Republican,Georgetown University(BA)\nWillamette Universi...,Georgetown University,No,Washington,DC,20057-0001,38.908809,-77.073463,Private nonprofit,7211,0.5813,62179
3,Alaska,Dan Sullivan,Republican,Harvard University(BA)\nGeorgetown University(...,Harvard University,Yes,Cambridge,MA,02138,42.374429,-71.118177,Private nonprofit,7236,0.4527,59950
4,Arizona,John McCain,Republican,United States Naval Academy (BS),United States Naval Academy,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Now to our analyses! 
senator_univs["CONTROL"] = senator_univs["CONTROL"].fillna("Military")
senator_univs.groupby(["DeVos", "CONTROL"]).count()[["Name"]]

# Republicans went to more public schools, as a percentage, than Democrats

Name
DeVos CONTROL                
No    Military              1
      Private nonprofit    34
      Public               15
Yes   Military              2
      Private nonprofit    26
      Public               22

In [46]:
senator_univs.groupby(["DeVos", "Party"]).median()

# Republicans also went to whiter schools, but paid less
# Lot of land-grant public universities, maybe?

LATITUDE  LONGITUDE     UGDS  UGDS_WHITE  COSTT4_A
DeVos Party                                                           
No    Democratic   40.727105 -79.953926   7018.0     0.53290   59730.0
      Independent  42.746625 -79.945573   4956.5     0.46475   64133.5
      Republican   41.749344 -76.117403   4746.5     0.68655   60864.5
Yes   Republican   38.557201 -89.539377  13117.0     0.71380   25918.0

In [ ]:
# The story is that, ...